<a href="https://colab.research.google.com/github/emkafie/Machine-Learning/blob/main/P3_JS09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Praktikum 3**

## **Langkah 1**

In [1]:
# ==========================================
# IMPORT LIBRARY
# ==========================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [4]:
# ==========================================
# LANGKAH 1: LOAD DATA
# ==========================================
# Menggunakan encoding 'latin-1' karena dataset SMS sering mengandung karakter spesial
try:
    df = pd.read_csv('spam.csv', encoding='latin-1')
    print("Data berhasil di-load!")
    print(df.head())
except FileNotFoundError:
    print("File 'spam.csv' tidak ditemukan. Pastikan file ada di direktori yang sama.")
    # Keluar dari program jika file tidak ada untuk mencegah error selanjutnya
    exit()

Data berhasil di-load!
     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


## **Langkah 2**

In [5]:
# ==========================================
# LANGKAH 2: PREPROCESSING
# ==========================================

# 1. Drop 3 kolom terakhir yang tidak berguna (Unnamed)
df = df.drop(df.iloc[:,2:], axis=1)

# 2. UBAH NAMA KOLOM (Penting agar langkah selanjutnya berjalan)
# Mengubah v1 menjadi 'Labels' dan v2 menjadi 'SMS'
df.rename(columns={'v1': 'Labels', 'v2': 'SMS'}, inplace=True)

# 3. Inspeksi Data
print("\n--- Inspeksi Data ---")
print(df['Labels'].value_counts())
print("\nDeskripsi Data:")
print(df.describe())

# 4. Encoding Label
# spam = 1, ham = 0
new_labels = {
    'spam': 1,
    'ham': 0
}
df['Labels'] = df['Labels'].map(new_labels)

# 5. Memisahkan Fitur (X) dan Label (y)
X = df['SMS'].values
y = df['Labels'].values

print("\n--- Contoh Data setelah Preprocessing ---")
print(df.head())


--- Inspeksi Data ---
Labels
ham     4825
spam     747
Name: count, dtype: int64

Deskripsi Data:
       Labels                     SMS
count    5572                    5572
unique      2                    5169
top       ham  Sorry, I'll call later
freq     4825                      30

--- Contoh Data setelah Preprocessing ---
   Labels                                                SMS
0       0  Go until jurong point, crazy.. Available only ...
1       0                      Ok lar... Joking wif u oni...
2       1  Free entry in 2 a wkly comp to win FA Cup fina...
3       0  U dun say so early hor... U c already then say...
4       0  Nah I don't think he goes to usf, he lives aro...


## **Langkah 3**

In [6]:
# ==========================================
# LANGKAH 3: EKSTRAKSI FITUR (BAG OF WORDS)
# ==========================================

# Split data training dan testing (80:20)
# random_state=50 agar hasil konsisten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

# Inisiasi CountVectorizer
bow = CountVectorizer()

# Fitting dan Transform pada X_train
# Kita mempelajari kosakata (vocabulary) HANYA dari data training
X_train = bow.fit_transform(X_train)

# Transform pada X_test
# Kita menggunakan kosakata yang dipelajari dari X_train untuk mengubah X_test.
# Jangan pernah melakukan fit() pada data testing untuk mencegah Data Leakage.
X_test = bow.transform(X_test)

# Cek dimensi fitur
# Menggunakan get_feature_names_out() karena get_feature_names() sudah deprecated di versi baru sklearn
print(f"\nJumlah Fitur (Vocabulary): {len(bow.get_feature_names_out())}")
print(f"Dimensi Data Training: {X_train.shape}")


Jumlah Fitur (Vocabulary): 7727
Dimensi Data Training: (4457, 7727)


## **Langkah 4**

In [7]:
# ==========================================
# LANGKAH 4: TRAINING DAN EVALUASI MODEL
# ==========================================

# Inisiasi Multinomial Naive Bayes
mnb = MultinomialNB()

# Latih model dengan data training
mnb.fit(X_train, y_train)

# Prediksi data training
y_pred_train = mnb.predict(X_train)

# Prediksi data testing
y_pred_test = mnb.predict(X_test)

# Evaluasi Akurasi
acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)

print(f'\n--- Hasil Evaluasi Multinomial Naive Bayes ---')
print(f'Akurasi Data Train : {acc_train}')
print(f'Akurasi Data Test  : {acc_test}')


--- Hasil Evaluasi Multinomial Naive Bayes ---
Akurasi Data Train : 0.9946152120260264
Akurasi Data Test  : 0.9775784753363229


CountVectorizer (Bag of Words): Teknik ini mengubah teks menjadi matriks angka. Setiap kata unik dalam seluruh data training dianggap sebagai satu fitur. Jika sebuah SMS mengandung kata tersebut, nilainya bertambah. Inilah mengapa disebut data multinomial (berbasis cacahan/frekuensi).

Mengapa fit_transform hanya di Train? Ini adalah konsep Data Leakage. Jika kita melakukan fit pada data Test, model akan "mengintip" kata-kata yang ada di masa depan (data ujian). Kita harus berpura-pura bahwa data Test adalah data baru yang belum pernah dilihat sistem sebelumnya, sehingga kita hanya menggunakan kosakata yang dipelajari dari Train.

Hasil Akurasi Tinggi: Hasil akurasi ~97% pada data test menunjukkan bahwa algoritma Naive Bayes sangat efektif untuk klasifikasi teks sederhana seperti Spam vs Ham, karena kata-kata dalam spam (misal: "win", "free", "prize") cenderung memiliki probabilitas kemunculan yang sangat berbeda dibandingkan SMS biasa.